In [1]:
from shenfun import *
import sympy as sp
import matplotlib.pyplot as plt
config['basisvectors'] = 'normal'

Define the position vector

In [2]:
t, theta = psi = sp.symbols('x,y', real=True)
r = (t+3)/2
rv = (r*sp.cos(theta), r*sp.sin(theta))

Create function spaces with curvilinear coordinates.

In [3]:
family = 'C'

N = 50
S1 = FunctionSpace(N, family, bc=(0, 0))
V2 = FunctionSpace(N+2, family, basis='Phi2')
T = TensorProductSpace(comm, (S1, S1), coordinates=(psi, rv, sp.Q.positive(t+3)))
V = TensorProductSpace(comm, (V2, V2), coordinates=(psi, rv, sp.Q.positive(t+3)))
u = TrialFunction(T)
v = TestFunction(V)

Print the Laplace operator in the new coordinates

In [4]:
from IPython.display import Math
du = div(grad(u))
Math((du*((t+3)/2)**2).tolatex(symbol_names={t: 't', theta: '\\theta'}))

<IPython.core.display.Math object>

Create a manufactured solution and compute the corresponding right hand side. 

In [5]:
ue = sp.sin(2*sp.pi*theta)*(r-1)*(r-2)*sp.cos(r*sp.pi)
fe = (div(grad(u))).tosympy(basis=ue, psi=psi)

Assemble left hand side matrix and right hand side vector.

In [6]:
f_hat = Function(V, buffer=fe)
A = inner(div(grad(u)), v*(t+3))

Solve problem and check that it is solved correctly.

In [7]:
sol = la.Solver2D(A)
u_hat = Function(T)
u_hat = sol(inner(v*(t+3), f_hat), u_hat)
uq = Array(T, buffer=ue)
assert np.sqrt(inner(1, (u_hat.backward()-uq)**2)) < 1e-6

Verify that assembled matrices above correspond to Eq. (4.33) in the paper. Note that the matrix computed with `Lmat` corresponds to the matrix in Eq. (4.10). 

In [8]:
from shenfun.jacobi.recursions import *

alfa = {'L': 0, 'C': -half}[family]
M = N-2
K = S1.stencil_matrix()
K.shape = (M, N)
A0 = Lmat(2, 2, 0, M, N, alfa, alfa, cn) + 6*Lmat(2, 1, 0, M, N, alfa, alfa, cn) + 9*SparseMatrix({2: 1}, (M, N)).diags()
A00 = extract_diagonal_matrix(A0*K.diags().T)
m = extract_diagonal_matrix(A00.diags()-A[0].mats[0].diags())
assert dict(m) == {}

A1 = Lmat(2, 1, 1, M, N, alfa, alfa, cn) + 3*Lmat(2, 0, 1, M, N, alfa, alfa, cn)
A11 = extract_diagonal_matrix(A1*K.diags().T)
m1 = extract_diagonal_matrix(A11.diags()-A[1].mats[0].diags())
assert dict(m1) == {}

A2 = extract_diagonal_matrix(Lmat(2, 0, 2, M, N, alfa, alfa, cn)*K.diags().T)
m2 = extract_diagonal_matrix(A2.diags()-A[2].mats[0].diags())
assert dict(m2) == {}

If you make it here with no error, then this means the matrices computed with quadrature above (in list `A`) are equal to the explicit matrix computed with `Lmat`, and described explicitly in the paper.